In [1]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from matplotlib.colors import LogNorm
from matplotlib import rc
from numpy import inf
import os
import sys
from IPython import display


#-- this selects the first GPU
os.environ['CUDA_VISIBLE_DEVICES']="0"

from os import listdir

from pickle import dump
from pickle import load

import uproot3

import matplotlib as mpl

from datetime import datetime
import subprocess


In [2]:

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.losses import Loss, BinaryCrossentropy
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


from tensorflow.python.client import device_lib

from pickle import dump


#has_gpu = False

has_gpu = True



from tensorflow.keras.callbacks import EarlyStopping
earlystopping = EarlyStopping(patience=20,
               verbose=True,
               restore_best_weights=True)

import os

print(tf.config.list_physical_devices())

if has_gpu :
    #os.environ['CUDA_VISIBLE_DEVICES']="0"
    physical_devices = tf.config.list_physical_devices('GPU') 
    tf.config.experimental.set_memory_growth(physical_devices[0], True)


    print("\n\n printing physical devbices from tf.config.list_physical_devices\n\n")
    print(physical_devices)

    devices = device_lib.list_local_devices()

    print("\n\n printing devices from device_lib.list_local_devices\n\n")
    print(devices)


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


 printing physical devbices from tf.config.list_physical_devices


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


 printing devices from device_lib.list_local_devices


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12367826736339611945
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 23048290304
locality {
  bus_id: 1
  links {
  }
}
incarnation: 1649931942733911106
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:41:00.0, compute capability: 8.6"
xla_global_id: 416903419
]


2023-08-14 10:41:25.129639: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-08-14 10:41:25.147566: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-08-14 10:41:25.147800: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-08-14 10:41:25.150613: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-08-14 10:41:25.150799: I tensorflow/compiler/xla/stream_executo

## Loss function for simple single classifier

In [3]:
bce = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def custom_loss_simple(y_true, y_pred):
    particle_level = y_pred
    loss = bce(y_true,particle_level)  
    #tf.print("custom_loss_simple : loss ", loss)
    #tf.print("custom_loss_simple :  y_true ", y_true, " y_pred ", y_pred )
    return loss

## Loss function for two classifier model, set up for the consistency test (only uses first classifier in loss)

In [4]:
bce = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def custom_loss(y_true, y_pred):
    particle_level = y_pred[:,0]
    detector_level = y_pred[:,1]
    loss = bce(y_true,particle_level) 
    #loss = 0.5*bce(y_true,particle_level) + 0.5*bce(y_true,detector_level) # test for when feeding in 2xpi
    #tf.print( "custom_loss : loss ", loss)
    #tf.print( "custom_loss : y_true ", y_true, " y_pred ", y_pred )
    #loss = bce(y_true,detector_level) 
    #loss = bce(y_true,particle_level)  + hp_mse_scale * tf.reduce_mean(tf.square(particle_level - detector_level)) 
    return loss

In [5]:
test_data = np.c_[
    [[0.1,0.2,0.3,0.4,0.5,0.6],
     [0.11,0.12,0.13,0.14,0.15,0.16]]
]

In [6]:
test_data

array([[0.1 , 0.2 , 0.3 , 0.4 , 0.5 , 0.6 ],
       [0.11, 0.12, 0.13, 0.14, 0.15, 0.16]])

In [7]:
test_y = np.c_[ 
    [[1],
     [0] ]
]

In [8]:
test_y

array([[1],
       [0]])

## Test of simple single classifier model

In [9]:
n = 3  # Number of features in input.

inputs = Input(shape=(2*n,))

classifier_nn = Sequential()
classifier_nn.add( Dense( 1, activation="linear", input_dim=n) )


particle_inputs_model = classifier_nn( inputs[:,0:n] )


output = particle_inputs_model


mymodel = Model( inputs=[inputs], outputs=[output])

learning_rate_setval=1e-3
opt_adam = tf.keras.optimizers.Adam(learning_rate=learning_rate_setval, amsgrad=False )

mymodel.compile( loss=custom_loss_simple, optimizer=opt_adam, metrics=['accuracy'] )


2023-08-14 10:41:25.577808: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-08-14 10:41:25.578026: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-08-14 10:41:25.578192: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-08-14 10:41:25.578392: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-08-14 10:41:25.578560: I tensorflow/compiler/xla/stream_executo

In [10]:
weights = mymodel.get_weights()
print(weights)

[array([[-0.35084003],
       [-0.24733382],
       [-0.01587522]], dtype=float32), array([0.], dtype=float32)]


In [11]:
weights[0][0][0] = 1
weights[0][1][0] = 1
weights[0][2][0] = 1
print(weights)

[array([[1.],
       [1.],
       [1.]], dtype=float32), array([0.], dtype=float32)]


In [12]:
mymodel.set_weights(weights)

In [13]:
mymodel.get_weights()

[array([[1.],
        [1.],
        [1.]], dtype=float32),
 array([0.], dtype=float32)]

In [33]:
output=mymodel(test_data)
print(output)

tf.Tensor(
[[0.6       ]
 [0.35999998]], shape=(2, 1), dtype=float32)


In [25]:
checksum0 = test_data[0][0] + test_data[0][1] + test_data[0][2]
checksum1 = test_data[1][0] + test_data[1][1] + test_data[1][2]
print(" Checksums ", checksum0, " and ", checksum1)

 Checksums  0.6000000000000001  and  0.36


In [15]:
mymodel.evaluate(test_data,test_y)

1/1 [==============================] - 0s 137ms/step - loss: 0.6634 - accuracy: 1.0000


[0.6633741855621338, 1.0]

In [45]:
bce(test_y,output)

<tf.Tensor: shape=(), dtype=float32, numpy=0.6633742>

In [48]:
custom_loss_simple(test_y,output)

<tf.Tensor: shape=(), dtype=float32, numpy=0.6633742>

In [54]:
output

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[0.6       ],
       [0.35999998]], dtype=float32)>

In [59]:

term_0 = test_y * np.log(output + 1e-7)
term_1 = (1-test_y) * np.log(1-output + 1e-7)
bce_calc = -np.mean(term_0+term_1, axis=0)
print(" calculated BCE :  ", bce_calc)

 calculated BCE :   [0.47855616]


### Why does the by-hand calculation of BCE not agree with the tf function?

## Test of two classifier model

In [16]:
n = 3  # Number of features in input.

inputs = Input(shape=(2*n,))

classifier_nn = Sequential()
classifier_nn.add( Dense( 1, activation="linear", input_dim=n) )


particle_inputs_model2 = classifier_nn( inputs[:,0:n] )
detector_inputs_model2 = classifier_nn( inputs[:,n:] )

output2 = tf.stack( [particle_inputs_model2, detector_inputs_model2], axis=1)


mymodel2 = Model( inputs=[inputs], outputs=[output2])

learning_rate_setval=1e-3
opt_adam = tf.keras.optimizers.Adam(learning_rate=learning_rate_setval, amsgrad=False )

mymodel2.compile( loss=custom_loss, optimizer=opt_adam, metrics=['accuracy'] )


In [17]:
weights2=mymodel2.get_weights()
print(weights2)

[array([[-0.6719996],
       [ 0.9788033],
       [ 1.1561435]], dtype=float32), array([0.], dtype=float32)]


In [18]:
weights2[0][0][0] = 1
weights2[0][1][0] = 1
weights2[0][2][0] = 1
print(weights2)

[array([[1.],
       [1.],
       [1.]], dtype=float32), array([0.], dtype=float32)]


In [19]:
mymodel2.set_weights(weights2)

In [20]:
mymodel2.get_weights()

[array([[1.],
        [1.],
        [1.]], dtype=float32),
 array([0.], dtype=float32)]

In [21]:
output2=mymodel2(test_data)
print(output2)

tf.Tensor(
[[[0.6       ]
  [1.5       ]]

 [[0.35999998]
  [0.45000002]]], shape=(2, 2, 1), dtype=float32)


In [27]:
checksum0a = test_data[0][0] + test_data[0][1] + test_data[0][2]
checksum1a = test_data[1][0] + test_data[1][1] + test_data[1][2]
print(" Checksums for network a ", checksum0a, " and ", checksum1a)

checksum0b = test_data[0][3] + test_data[0][4] + test_data[0][5]
checksum1b = test_data[1][3] + test_data[1][4] + test_data[1][5]
print(" Checksums for network b ", checksum0b, " and ", checksum1b)

 Checksums for network a  0.6000000000000001  and  0.36
 Checksums for network b  1.5  and  0.45000000000000007


In [28]:
mymodel2.evaluate(test_data,test_y)

1/1 [==============================] - 0s 17ms/step - loss: 0.6634 - accuracy: 1.0000


[0.6633741855621338, 1.0]

In [47]:
custom_loss(test_y,output2)

<tf.Tensor: shape=(), dtype=float32, numpy=0.6633742>